<table align="center"><tr><th> <div style="width:600px"> <h4>AGH University of Krakow <br><br> Faculty of Computer Science, Electronics and Telecommunication <br><br> Institute of Electronics </h4></div></th><th> <div style="width:200px"><img src="./img/logo_agh.png" width="68" height="136"/></div> </th></tr></table>

---

<div style="text-align:center"><h3>CUSTOM SYSTEM DESIGN IN FPGA LABORATORY</h3></div>
<br>
<div style="text-align:center"><h1> Sprawozdanie z projektu </h1></div>
<br>

> **NOTE:** This tutorial shows how to start your adventure with the PYNQ system using the [AMD-Xilinx KRIA KV260](https://www.amd.com/en/products/system-on-modules/kria/k26/kv260-vision-starter-kit.html) development platform.<br>After rebuilding the project in Vivado, all examples can be run on any FPGA SOC platform like [Zedboard](https://digilent.com/reference/programmable-logic/zedboard/start), [Zybo](https://digilent.com/reference/programmable-logic/zybo/start).


## Contents

* [Connection and communication with the development board](#Connection-and-communication-with-the-development-board)
* [Cel projektu](#Cel-projektu)
* [Wprowadzenie do tematu projektu](#Wprowadzenie-do-tematu-projektu)
* [Using a new overlay with PYNQ](#Using-a-new-overlay-with-PYNQ)

<div style="text-align:right"><h5> ver 0.2.1 </h5></div><br>

# Cel projektu

Celem projektu jest stworzenie sprzętowej implementacji algorytmu sortowania bitconic
z wykorzystaniem układu Zynq, który łączy elastyczność procesora ARM z mocą programowalnej logiki FPGA.
Projekt ten ma na celu przeniesienie obliczeń z warstwy programowej do sprzętowej. 
Zynq pozwala na stworzenie dedykowanej architektury równoległej, która idealnie pasuje do natury bitonic sort.
Kluczowe elementy algorytmu zostaną odwzorowane jako struktury sprzętowe pracujące synchronicznie i deterministycznie. 
W projekcie przewiduje się integrację części logicznej z systemem operacyjnym zainstalowanym na płytce rozwojowej Kria KV260,
co umożliwi łatwą konfigurację i kontrolę procesu sortowania.
Efektem końcowym będzie akcelerator sprzętowy, 
który może znaleźć zastosowanie w systemach wymagających szybkiego przetwarzania danych.

# Wprowadzenie do tematu projektu

## Wstęp
Bitonic Sort to algorytm sortowania oparty na tworzeniu tzw. sekwencji bitonicznych (ciągów, które najpierw rosną, a potem maleją (lub odwrotnie)). Algorytm ten w kolejnych etapach dzieli i porównuje elementy, aż uzyska w pełni posortowany ciąg(proces składa się z 6 stanów). 
Bitconic sort dzięki swojej strukturze dobrze nadaje się do implementacji sprzętowej i potokowej(szczególnie w FPGA).
Algorytm ten działa z czasem O(log² n).

## Zasada działania algorytmu
Zasada działania algorytmu opiera się o dwa główne bloki funkcjonalne - minMax oraz Maxmin z których składa się główny moduł sortujący.
Ich zadaniem jest porównywanie dwóch 8-bitowych liczb i przypisywanie mniejszej i większej z nich do odpowiednich wyjść. Różnica między nimi polega na kierunku sortowania:

1. maxMin - moduł który jest wykorzystywany w etapach sortowania, w których tworzone są sekwencje rosnące.

2. minMax - moduł który jest wykorzystywany w etapach sortowania, w których tworzone są sekwencje malejące.

Takie podejście pozwala elastycznie budować sekwencje bitoniczne, w których kluczowe jest odpowiednie przemieszczanie danych zgodnie z ustalonym wzorcem porównań i zamian.


| ![blok_komparator_1](img/blok_komparator_1.png) | ![blok_komparator_2](img/blok_komparator_1.png) |
|------------------------------|------------------------------|

Mając tak zaprojektowane moduły zaprojektowano główny moduł układu sortujacego zgodnie z schematem

![Algorytm](img/algorytm.png "Bitonic Sort Diagram")


# Projekt systemu sortującego - integracja z AXI 

Kolejnym etapem projektu była integracja zaprojektowanego algorytmu z płytką deweloperską Kria kv260. Połączenie tych dwóch rzeczy możliwe było dzięki wykorzystaniu magistrali AXI Lite, dzięki której możliwa jest komunikacja zaprojektowanego IP Core z Zynq UltraScale+.

Zaprojektowany z wykorzystaniem środowiska Vivado układ przedstawiono poniżej.
![Diagram sortowania bitonicznego](img/diagram.jpeg "Bitonic Sort Diagram")

# Testy zaprojektowanego systemu
## Projekt systemu sortującego - symulacja behawioralna w Vivado

Pierwszym wykonanym testem zaraz po zaprojektowaniu systemu była symualcja behawioralna w Vivado, jej wynik przedstawiono poniżej

![Wynik symulacji](img/przebieg.jpeg "Symulation result")

## Projekt systemu sortującego - symulacja behawioralna w języku Python

Drugim wykonanym testem była implementacja algorytmu bitconic w języku Python. Dzięki takiemu podejściu można łatwo porównać czy sprzętowa implementacja algorytmu przyniosła pozytywne efekty takie jak np. przyśpieszenie obliczeń itp. Zaprojektowany algorytm w języku Python przedstawiono poniżej.

In [ ]:
def compare_and_swap(arr, i, j, direction):
    if (direction == 1 and arr[i] > arr[j]) or (direction == 0 and arr[i] < arr[j]):
        arr[i], arr[j] = arr[j], arr[i]

def bitonic_merge(arr, low, cnt, direction):
    if cnt > 1:
        k = cnt // 2
        for i in range(low, low + k):
            compare_and_swap(arr, i, i + k, direction)
        bitonic_merge(arr, low, k, direction)
        bitonic_merge(arr, low + k, k, direction)

def bitonic_sort_recursive(arr, low, cnt, direction):
    if cnt > 1:
        k = cnt // 2
        bitonic_sort_recursive(arr, low, k, 1)          
        bitonic_sort_recursive(arr, low + k, k, 0)      
        bitonic_merge(arr, low, cnt, direction)

def bitonic_sort(arr, ascending=True):
    n = len(arr)
    if n & (n - 1) != 0:
        raise ValueError("Array length should be power of 2.")
    bitonic_sort_recursive(arr, 0, n, 1 if ascending else 0)
    return arr


a = [3, 7, 4, 8, 6, 2, 1, 5]
bitonic_sort(a)

## Zysk obliczeniowy algorytmu sortującego
W celu weryfikacji uzyskanych efektów, szczególnie czasu trwania algorytmu dla zadanych ilosci danych przpeorwdzono serię pomiarów których wynik przedstawiono poniżej.

| liczba danych | Python [s] | Bitconic [s] | Zysk obliczeniowy[s] |
|---------------|------------|--------------|----------------------|
| 100           | 0.01397    | 0.0139       | 7E-05                |
| 1000          | 0.1297     | 0.124        | 0.0057               |
| 10000         | 1.27       | 1.2          | 0.07                 |
| 100000        | 12.62      | 12.3         | 0.32                 |
| 200000        | 25.11      | 24.06        | 1.05                 |


![zysk_obliczen](img/zysk_boliczen.jfif "Bitonic Sort Diagram")

# Connection and communication with the development board

Część projektu zaczerpnięta z wprowadzenia do "PYNQ - Introduction" stworzonego przez Pana Sebastiana Koryciaka.


> **STEP 1:** Connect cables to the KRIA
>
>> A. Insert the microSD card to the J11<br>
>> B. Connect micro-USB cable to J4<br>
>> C. Connect RJ45 cable<br>
>> D. Connect power supply to J12<br>
>
> **WARNING:** After connecting to power supply the fan located at the KRIA board will start at full speed. It will slow down after correct boot of Ubuntu OS, in approx. 3 minuts.

<img src="./img/kria_plugs.png" width="300"/>


> **STEP 2:** Connect to the network
>
> PYNQ is working most efficient if it has access to the Internet. If available, you should connect your board to a network or router with Internet access. This will allow you to update your board and easily install new packages. There are two options:
>
>> LEFT option: Connect the KRIA with RJ45 directly to the PC (embedded network card, or dongle **USB<->Ethernet** included in the lab box)<br>
>> RIGHT option: Connet the KRIA to the router<br>

<img src="./img/kria_setup_network.png" width="800"/>

> You will need to have an Ethernet port available on your computer, and you will need to have permissions to configure your network interface. With a direct connection, you will be able to use PYNQ, but unless you can bridge the Ethernet connection to the board to an Internet connection on your computer, your board will not have Internet access. You will be unable to update or load new packages without Internet access.<br>
>
>> In the example below **Ethernet** is PC connection to LAN, and **Ethernet 3** is connection with KRIA. Use **View Network Connections** to set **Internet Connetion Sharing** option.<br>In case of no Received Bytes on **Ethernet 3** network, the best solution is to *disable and enable* network sharing in the **Ethernet** Properties tab.

<img src="./img/setup_ethernet.png" width="800"/>

> **STEP 3:** Open a USB Serial Terminal
>
>  You can use the terminal to check the network connection of the board. [PuTTY](https://www.putty.org/) is one application that can be used. To open a terminal, you will need to know the COM port for the board. Use 115200 baud rate. To login use data:
>
>> username: ubuntu<br>
>> password: kriasdup<br>
>
> You can check the HOSTNAME and IP address of the board using *ifconfig*. 
> In this example: 
>
>> HOSTNAME: kriaSDUP-0<br>
>> IPaddress: 192.168.137.227

<img src="./img/kria_putty.png" width="500"/>

> **STEP 4:** Connecting to Jupyter Notebook (LAB)
>
> Once your board is setup, to connect to Jupyter Notebooks open a web browser on your PC and navigate to:
>
>> [HOSTNAME:9090/lab](HOSTNAME:9090/lab) or <br>
>> [IPaddress:9090/lab](IPaddress:9090/lab)<br>
>
> To login use data:
>
>> password: xilinx<br>
>
> You can use drag-and-drop to copy files to the board.<br>
> To download file from the board use option File -> Download.

<img src="./img/kria_jupyter.png" width="600"/>

<div class="alert alert-success"><strong>NOTE:</strong> Starting from now you can work on a copy of this document.<br> Create your work folder (ex. /sdup/student) and copy to it extracted archive (T8_pynq_notebook.zip from UPEL webpage) which consists of jupyter notebook file and a folder with images.<br>Then just open the notebook file on KRIA and go the the next step.</div>

### Instantiate the Overlay

# Using a new overlay with PYNQ
Załadowanie modułu:

In [99]:
from pynq import Overlay
from time import sleep
import random

kv260_sdup_ov = Overlay("bitconic_v2.xsa")

Przetestowanie załączenia:

In [68]:
kv260_sdup_ov

**Po poprawnym załączeniu modułu** należy przetestować podstawowe działanie

In [154]:
kv260_sdup_ov.bitconic_0.write(0, 0x04_03_02_01)
kv260_sdup_ov.bitconic_0.write(4, 0x05_06_07_08)
kv260_sdup_ov.bitconic_0.write(4*4, 1)


i0 = kv260_sdup_ov.bitconic_0.read(4*0)
i1 = kv260_sdup_ov.bitconic_0.read(4*1)
valid = kv260_sdup_ov.bitconic_0.read(4*4)
print(f"i0: {i0}, i1: {i1}, valid: {valid}")

i0: 67305985, i1: 84281096, valid: 1


In [155]:
o0 = kv260_sdup_ov.bitconic_0.read(4*2)
o1 = kv260_sdup_ov.bitconic_0.read(4*3)
valid = kv260_sdup_ov.bitconic_0.read(4*5)
print(f"o0: 0x{o0:X}, o1: 0x{o1:X}, valid: {valid}")

o0: 0x1020304, o1: 0x5060708, valid: 0


In [156]:
o0 = kv260_sdup_ov.bitconic_0.read(4*2)
o1 = kv260_sdup_ov.bitconic_0.read(4*3)
valid = kv260_sdup_ov.bitconic_0.read(4*5)
print(f"o0: 0x{o0:X}, o1: 0x{o1:X}, valid: {valid}")

o0: 0x1020304, o1: 0x5060708, valid: 0


Jak widać układ sortujący pracuje poprawnie.

**Funkcje dodatkowe, poprawiające czytelność danych**

In [157]:
number = 8

def genRandom():
    L = range(256)
    
    tab = [random.choice(L) for _ in range(number)]
    print(f"Input:  {tab}")
    i0 = 0
    i1 = 0
    for i in range(number//2):
        i0 = (i0 << 8) + tab[i]
        i1 = (i1 << 8) + tab[i + number//2]
    return i0, i1

def decode(o0, o1, valid):
    ou0 = []
    ou1 = []
    for i in range(number//2):
        ou0.append(o0 & 0xFF)
        o0 = o0 >> 8

        ou1.append(o1 & 0xFF)
        o1 = o1 >> 8
    ou0.reverse()
    ou1.reverse()
    tab = [*ou0, *ou1]
    print(f"Output: {tab}")

In [138]:
def toSort(i0, i1):
    kv260_sdup_ov.bitconic_0.write(4*0, i0)
    kv260_sdup_ov.bitconic_0.write(4*1, i1)
    kv260_sdup_ov.bitconic_0.write(4*4, 1)
    
def fromSort():
    o0    = kv260_sdup_ov.bitconic_0.read(4*2)
    o1    = kv260_sdup_ov.bitconic_0.read(4*3)
    valid = kv260_sdup_ov.bitconic_0.read(4*5)
    return o0, o1, valid

In [159]:
i0, i1 = genRandom()
decode(i0, i1, 0)

Input:  [105, 207, 83, 136, 21, 151, 108, 133]
Output: [105, 207, 83, 136, 21, 151, 108, 133]


In [160]:
toSort(i0, i1)
o0, o1, valid = fromSort()
decode(o0, o1, valid)

Output: [21, 83, 105, 108, 133, 136, 151, 207]


## Test dla losowych wektorów

In [158]:
for _ in range (20):
    i0, i1 = genRandom()
    toSort(i0, i1)
    o0, o1, valid = fromSort()
    decode(o0, o1, valid)

Input:  [35, 230, 196, 188, 22, 7, 225, 233]
Output: [7, 22, 35, 188, 196, 225, 230, 233]
Input:  [9, 187, 60, 132, 23, 123, 71, 22]
Output: [9, 22, 23, 60, 71, 123, 132, 187]
Input:  [26, 222, 96, 141, 50, 83, 144, 101]
Output: [26, 50, 83, 96, 101, 141, 144, 222]
Input:  [220, 71, 231, 8, 11, 63, 69, 30]
Output: [8, 11, 30, 63, 69, 71, 220, 231]
Input:  [234, 131, 15, 211, 236, 32, 166, 197]
Output: [15, 32, 131, 166, 197, 211, 234, 236]
Input:  [149, 83, 186, 118, 45, 131, 162, 55]
Output: [45, 55, 83, 118, 131, 149, 162, 186]
Input:  [112, 122, 182, 61, 188, 53, 188, 209]
Output: [53, 61, 112, 122, 182, 188, 188, 209]
Input:  [94, 222, 198, 12, 50, 21, 156, 126]
Output: [12, 21, 50, 94, 126, 156, 198, 222]
Input:  [161, 145, 42, 247, 36, 242, 210, 114]
Output: [36, 42, 114, 145, 161, 210, 242, 247]
Input:  [149, 182, 26, 96, 45, 190, 182, 113]
Output: [26, 45, 96, 113, 149, 182, 182, 190]
Input:  [126, 184, 16, 47, 141, 24, 63, 240]
Output: [16, 24, 47, 63, 126, 141, 184, 240]
Inpu